In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:92% !important; }</style>"))

%load_ext autoreload
%autoreload 2
import sys, os
from os.path import expanduser
## actions required!!!!!!!!!!!!!!!!!!!! change your folder path 
path = "~/Documents/G3/MA-prediction"
path = expanduser(path)
sys.path.append(path)

import data_science_MA_kit as dsk
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import re
# import wrds

pd.options.mode.chained_assignment = None

# Data preprocessing 3: Variable Transformation

In this notebook we will do the following:

- Create `amend` and `choice` binary variables.
- Create competing group number and competing status code.
- Create deal value adjusted by CPI index.
- Create new payment type.
- Extract cash and stock terms from consideration.

## Load data

In [2]:
filepath = f"{path}/data/df_dates_corrected.h5"
df = pd.read_hdf(filepath)

# Create `amend` and `choice` binary variables

In [3]:
# create amend
dsk.insert_cols(df, 'valamend', 'amend', dsk.create_amend(df))

# create consid
dsk.insert_cols(df, 'consid', 'choice', dsk.create_choice(df))

# Create competing group number and status code

## Create competing group number

In [4]:
# change to 0/1
df.cha = df.cha.replace({'Yes':1, 'No':0})

In [5]:
# create competing deal group numbers
dsk.insert_cols(df, 'lbo', 'compete_group_no', dsk.create_compete_group_no(df))

# update cha. Some errors in the database
df.cha[~df.compete_group_no.isna()] = 1

# check error
if df.compete_group_no.eq(-1).any():
    print('Different group numbers in one group.')
if df.compete_group_no.value_counts().eq(1).any():
    print('Group of a single deal.')

## Create competing status code

In [6]:
# create competing status code
dsk.insert_cols(df, 'lbo', 'compete_statc_code', dsk.create_compete_status_code(df))

In [7]:
df.compete_statc_code.value_counts()

2.0    438
0.0    332
1.0     52
3.0      1
Name: compete_statc_code, dtype: int64

# Create announced year and month. Create `val` adjusted by CPI index

## Create announced year and month.
Create announced year and month.

In [8]:
# create announced year and month
df['ann_year'] = df.da.map(lambda x: x.year)     # useful later
df['ann_year_month'] = df.da.map(lambda x: pd.Period(f'{x.year}-{x.month}', 'M'))

## Create deal value adjusted by CPI index. Delete monthly period column.
CPI data is downloaded from FRED [website](https://fred.stlouisfed.org/series/CPIAUCSL). We set the value of Dec 2020 as the base, and inflate all the deal values beforehand by the CPI index.

In [9]:
html = "https://fred.stlouisfed.org/graph/fredgraph.csv?id=CPIAUCSL"
df_cpi = pd.read_csv(html)

In [10]:
cpi_ser = dsk.to_monthly_period_index(df_cpi, 'DATE').squeeze()
cpi_ser.head()

month
1947-01    21.48
1947-02    21.62
1947-03    22.00
1947-04    22.00
1947-05    21.95
Freq: M, Name: CPIAUCSL, dtype: float64

In [11]:
# create deal value adjusted by CPI index, base is '2020-12'
base = pd.Period('2020-12', 'M')
adj_factor = cpi_ser[base] / cpi_ser
# adjust deal value by CPI
dsk.insert_cols(df, 'val', 'val_adj_by_cpi', adj_factor[df.ann_year_month].values * df.val)

In [12]:
# unfortunately period dtype is not supported by hdf
df = df.drop(columns=['ann_year_month'])

# Create new payment type `consido`

In [13]:
dsk.extract_all_payment_types(df.consido)

array(['Amer. Depy. Receipt', 'American Depy Share', 'Assets', 'Bond',
       'Capital Infusion', 'Capital Stock', 'Cash', 'Cash Dividend',
       'Cash Only', 'Common L.P. Unit', 'Common Stock',
       'Common Stock Type A', 'Common Stock Type B',
       'Common Stock Type C', 'Common Stock Type H',
       'Conting. Value Right', 'Contingent Val Pref',
       'Convert. Debenture', 'Convert. Exch. Pfd.', 'Convert. Preferred',
       'Convert. Preferred A', 'Convert. Preferred B',
       'Convert. Preferred D', 'Convert. Securities',
       'Convert. Sub. Deb.', 'Convert. Sub. Note', 'Convertible Note',
       'Cum Cvt Exch Pfd', 'Cum. Convert. Pfd.', 'Cumulative Preferred',
       'Cvt Cum Pfd Ser A', 'Cvt. Promissory Note', 'Debenture', 'Debt',
       'Debt Reduction', 'Depositary Receipt', 'Depositary Share',
       'Earnout', 'Exch Redeemable Pfd', 'Exch. Sub. Deb.',
       'Junior Preferred Shs', 'L.P. Unit', 'Liabilities', 'Loan',
       'Ltd. Vot. Pfd ADR', 'Newly Issued Ord Sh',

In [14]:
lst_cash = ['Cash', 'Cash Only']
lst_stock = ['Common Stock', 'Common Stock Type A', 'Common Stock Type B', 'Common Stock Type C', 'Common Stock Type H', 'Newly Issued Ord Sh', 'Ordinary Share', ]

In [15]:
# # to be deal with later
# replace_lst_more = ['Amer. Depy. Receipt', 
#                     'American Depy Share',
#                     'Cash Dividend'
#                     'Common L.P. Unit',
#                    'Depositary Receipt',
#                     'Depositary Share',
#                    'L.P. Unit',
#                    'Unit']

In [16]:
# new consido
dsk.insert_cols(df, 'consido', 'consido_new', df.consido.map(lambda x: dsk.transform_payment_str(x, lst_cash, lst_stock)))
df.consido_new.value_counts()

Cash                     5105
Common Stock             2623
Cash and Common Stock    1469
No Cash or Stock          657
Name: consido_new, dtype: int64

# Extract cash and stock terms from consideration.

In [17]:
# convert `consid` column to a more readable format
df.consid = df.consid.map(dsk.convert_consid_to_readable, na_action='ignore')

In [18]:
# extract terms from `consid`
df['terms'] = dsk.extract_term_from_consid(df.consid)

In [19]:
#
cash_stock_terms = df['terms'].map(dsk.extract_cash_stock_from_term, na_action='ignore')
cash_stock_terms = pd.DataFrame(list(cash_stock_terms), index=df.index)

In [20]:
dsk.insert_cols(df, ['consid', 'consid', 'consid'], ['cash_term', 'stock_term', 'payment_type'], cash_stock_terms)

In [21]:
df.payment_type.value_counts()

Cash                                   4144
parse failed                           2062
Common Stock                           1998
Cash and Common Stock                   426
Common Stock, fixed dollar              217
Cash and Common Stock, fixed dollar      92
Name: payment_type, dtype: int64

# Save results

In [22]:
filepath = f"{path}/data/df_variable_transform.h5"

df.to_hdf(filepath, key = 'df', mode='w')

/var/folders/l5/gqn1skjd0v366h5y3kbms6lh0000gn/T/ipykernel_98117/2718949586.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['statc', 'one_day', 'aone_day', 'dao', 'da_corrected', 'da', 'de',
       'dateeffexp', 'dw', 'dr', 'definitive_agt', 'da_date', 'dateval',
       'dcom', 'dcomeff', 'last_trade_date', 'delist_date', 'att',
       'attitude_change_yn', 'initial_rec', 'tn_CRSP', 'tn', 'ttic_CRSP',
       'ttic', 'tcu_CRSP', 'tcu', 'texch', 'an_CRSP', 'an', 'apub',
       'atic_CRSP', 'atic', 'acu_CRSP', 'acu', 'anatc', 'aexch', 'cross',
       'ttf_macro_desc', 'ttf_mid_desc', 'atf_macro_desc', 'atf_mid_desc',
       'valamend', 'consid_struct_desc', 'payment_type', 'consid',
       'consido_new', 'consido', 'consids', 'tend', 'term', 'synop', 'hdate',
       'hosthprice', 'hval', 'hevent', 'hosthval', 'competecode', 'competeval',
       'lbo',